In [22]:
import os
import psycopg2 as ps
import pandas as pd

In [23]:
conn = ps.connect(
    host='localhost',
    port='5432',
    dbname='chinook',
    user='chinook',
    password='password')

# Revenue generated by the top 10 countries

In [24]:
df = pd.read_sql_query('''
    SELECT 
        c.country,
        COUNT(i.invoice_id) AS num_invoice,
        SUM(i.total) AS revenue,
        ROUND(AVG(i.total), 2) AS avg_value
    FROM customer c
    JOIN invoice i
    ON c.customer_id = i.customer_id
    GROUP BY c.country
    ORDER BY revenue DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\2867768354.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,country,num_invoice,revenue,avg_value
0,USA,91,523.06,5.75
1,Canada,56,303.96,5.43
2,France,35,195.10,5.57
3,Brazil,35,190.10,5.43
4,Germany,28,156.48,5.59
5,United Kingdom,21,112.86,5.37
6,Czech Republic,14,90.24,6.45
7,Portugal,14,77.24,5.52
8,India,13,75.26,5.79
9,Chile,7,46.62,6.66


### Key Takeaways:
1. High-Spending Markets: The Czech Republic and Chile stand out for their high average invoice values despite fewer transactions, indicating a concentrated but potentially lucrative market.
2. Large Markets with Lower Average Spend: The USA and Canada generate high revenues through a large number of transactions, but with lower average spending per invoice.
3. Balanced Markets: Countries like France, Brazil, and Germany show a balance between the number of transactions and average spending, making them stable markets with room for growth.

# Top 10 artists in the USA based on the number of tracks sold and the revenue they generated

In [25]:
df = pd.read_sql_query('''
	SELECT
		ar.name,
		SUM(il.quantity) AS num_tracks,
	  	SUM(il.unit_price * il.quantity) AS revenue
	FROM artist ar
	JOIN album al
	ON ar.artist_id = al.artist_id
	JOIN track t
	ON al.album_id = t.album_id
	JOIN invoice_line il
	ON t.track_id = il.track_id
	JOIN invoice i
	ON il.invoice_id = i.invoice_id
	WHERE i.billing_country = 'USA'
	GROUP BY ar.artist_id
	ORDER BY num_tracks DESC
	LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\2276041873.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,name,num_tracks,revenue
0,Iron Maiden,34,33.66
1,Metallica,27,26.73
2,U2,23,22.77
3,The Office,17,33.83
4,Deep Purple,15,14.85
5,Eric Clapton,14,13.86
6,Led Zeppelin,13,12.87
7,Amy Winehouse,13,12.87
8,Guns N' Roses,13,12.87
9,Tim Maia,12,11.88


### Key Takeaways:
1. Iron Maiden, Metallica, and U2 are the most popular in terms of both sales and revenue.
2. The Office has a lower number of tracks sold but generates the highest revenue per track, suggesting it might have premium content.
3. The remaining artists have similar performance metrics, with consistent but slightly lower sales and revenue figures.
### Potential Actions:
1. Promote High-Performing Artists: Focus on marketing and promotional efforts for top artists like Iron Maiden, Metallica, and U2 to further boost sales.
2. Leverage Premium Content: The Office’s high revenue per track indicates an opportunity to push premium content, which could be a strategy to increase overall revenue.
3. Explore Pricing Strategies: Analyzing the pricing strategies for these artists could provide insights into maximizing revenue, especially for those with lower track sales but higher revenue per track.

# Most popular music genre in each country by analyzing the number of tracks sold

In [26]:
df = pd.read_sql_query('''
	WITH unranked AS (
		SELECT
			i.billing_country AS country,
			g.name AS genre,
			SUM(il.quantity) AS num_tracks
		FROM genre g
		JOIN track t ON g.genre_id = t.genre_id
		JOIN invoice_line il ON t.track_id = il.track_id
		JOIN invoice i ON il.invoice_id = i.invoice_id
		GROUP BY i.billing_country, g.name
	),
	ranked AS (
		SELECT
			*,
			ROW_NUMBER() OVER(PARTITION BY country ORDER BY num_tracks DESC) AS rn
		FROM unranked
	)
	
	SELECT
		country,
		genre
	FROM ranked
	WHERE rn = 1
	ORDER BY num_tracks DESC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\1671279094.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,country,genre
0,USA,Rock
1,Canada,Rock
2,Brazil,Rock
3,France,Rock
4,Germany,Rock
5,United Kingdom,Rock
6,Portugal,Rock
7,India,Rock
8,Czech Republic,Rock
9,Spain,Rock


### Insights:
1. Global Dominance of Rock: Rock music is the most popular genre in almost all the countries listed, which may suggest that it has a broad international appeal.
2. Regional Preferences: The only exception in the dataset is Sweden, where Latin music takes the top spot. This could indicate a particular cultural or demographic trend that makes Latin music more popular there.

### Potential Actions:
1. Market Focus: Given the dominance of Rock, marketing efforts might focus on promoting Rock music, especially in the countries where it is already most popular.
2. Explore Regional Trends: In Sweden, Latin music’s popularity might indicate an opportunity to explore or expand into Latin music offerings, or to investigate why this genre is uniquely popular there.


#  List of tracks, along with their album titles and artist names, that have not been purchased in the last 6 months

In [27]:
df = pd.read_sql_query('''
    SELECT DISTINCT 
        t.name,
        al.title, 
        ar.name
    FROM track t
    JOIN invoice_line il ON t.track_id = il.track_id
    JOIN invoice i ON il.invoice_id = i.invoice_id
    JOIN album al ON t.album_id = al.album_id
    JOIN artist ar ON al.artist_id = ar.artist_id
    WHERE i.invoice_date < (SELECT MAX(invoice_date) FROM invoice) - INTERVAL '6' MONTH;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\2540680810.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,name,title,name
0,"Stay (Faraway, So Close!)",Zooropa,U2
1,Smokin' in The Boys Room,Motley Crue Greatest Hits,Mötley Crüe
2,Doutor,Cidade Negra - Hits,Cidade Negra
3,I Can't Explain,20th Century Masters - The Millennium Collecti...,Scorpions
4,"De Do Do Do, De Da Da Da",The Police Greatest Hits,The Police
...,...,...,...
1813,Me Deixas Louca,Elis Regina-Minha História,Elis Regina
1814,Last Night On Earth,Pop,U2
1815,Leave My Girl Alone,In Step,Stevie Ray Vaughan & Double Trouble
1816,Seu Balancê,Ao Vivo [IMPORT],Zeca Pagodinho


### Potential Actions:
1. Launch targeted marketing campaigns and promotions to increase visibility and encourage purchases of the tracks.
2. Collaborate with artists for new releases or exclusive content and send notifications to users about these tracks.
3. Review and adjust pricing strategies, improve track recommendations, and ensure availability across all relevant platforms.

# List of the top 10 customers by revenue, including their customer ID, name, country, number of invoices, and total revenue. This will help identify the most valuable customers based on their total spending.

In [28]:
df = pd.read_sql_query('''
    SELECT
        c.customer_id,
        CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
        c.country,
        COUNT(i.invoice_id) AS num_invoice,
        SUM(i.total) AS revenue 
    FROM customer c
    JOIN invoice i
    ON c.customer_id = i.customer_id
    GROUP BY c.customer_id
    ORDER BY revenue DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\3753801493.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,customer_id,customer_name,country,num_invoice,revenue
0,6,Helena Holý,Czech Republic,7,49.62
1,26,Richard Cunningham,USA,7,47.62
2,57,Luis Rojas,Chile,7,46.62
3,46,Hugh O'Reilly,Ireland,7,45.62
4,45,Ladislav Kovács,Hungary,7,45.62
5,28,Julia Barnett,USA,7,43.62
6,24,Frank Ralston,USA,7,43.62
7,37,Fynn Zimmermann,Germany,7,43.62
8,25,Victor Stevens,USA,7,42.62
9,7,Astrid Gruber,Austria,7,42.62


### Potential Actions:
1. Engage Top Customers: Develop targeted marketing campaigns or loyalty programs for these high-value customers to further increase their engagement and spending.
2. VIP program: Provide special offers or rewards based on their purchasing behavior and preferences. Consider offering exclusive content or discounts to retain these valuable customers and encourage continued purchases.
3. Customer Feedback: Reach out to these top customers for feedback to understand their preferences and improve your services or offerings.

# Turnover rate by genre (average revenue generated per track)

In [29]:
df = pd.read_sql_query('''
	WITH Sales AS (
	  SELECT g.genre_id, SUM(il.unit_price * il.quantity) AS total_sales
		FROM invoice_line il
		JOIN track t ON il.track_id = t.track_id
		JOIN genre g ON t.genre_id = g.genre_id
		GROUP BY g.genre_id
	),
	Inventory AS (
		SELECT g.genre_id, COUNT(t.track_id) AS num_tracks
		FROM track t
		JOIN genre g ON t.genre_id = g.genre_id
		GROUP BY g.genre_id
	)

	SELECT 
		g.name AS genre,
		s.total_sales,
		i.num_tracks,
		ROUND(s.total_sales/ i.num_tracks, 3) AS turnoverrate
	FROM Sales s
	JOIN Inventory i ON s.genre_id = i.genre_id
	JOIN genre g ON s.genre_id = g.genre_id
	ORDER BY turnoverrate DESC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\3258165506.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,genre,total_sales,num_tracks,turnoverrate
0,Sci Fi & Fantasy,39.80,26,1.531
1,Comedy,17.91,17,1.054
2,TV Shows,93.53,93,1.006
3,Bossa Nova,14.85,15,0.990
4,Science Fiction,11.94,13,0.918
5,Drama,57.71,64,0.902
6,Blues,60.39,81,0.746
7,Alternative & Punk,241.56,332,0.728
8,Metal,261.36,374,0.699
9,R&B/Soul,40.59,61,0.665


### Potential Actions:
1. Promote High Turnover Rate Genres: Focus marketing efforts on genres with the highest turnover rates, like Sci Fi & Fantasy and Comedy, to leverage their strong revenue potential per track.

2. Feature High Revenue Genres: Use genres with high total sales, such as Rock and Latin, in promotional activities to attract more customers despite their lower turnover rates.

3. Optimize Track Inventory: Review and potentially streamline inventory for genres with low turnover rates, like Heavy Metal and Electronica/Dance, to ensure focus on high-performing tracks.

4. Adjust Pricing Strategies: Consider revising pricing or bundling offers for genres with low turnover rates to increase revenue per track and improve overall performance.

5. Monitor and Analyze Trends: Continuously track turnover rates and sales data to identify trends and make informed adjustments to marketing, pricing, and inventory strategies.

# Monthly Sales Trends with Rolling Average

In [30]:
df = pd.read_sql_query('''
    WITH MonthlySales AS (
        SELECT
            EXTRACT(YEAR FROM i.invoice_date) AS year,
            EXTRACT(MONTH FROM i.invoice_date) AS month,
            SUM(il.unit_price * il.quantity) AS total_sales
        FROM invoice i
        JOIN invoice_line il ON i.invoice_id = il.invoice_id
        GROUP BY year, month
    ),
    SalesWithRollingAverage AS (
        SELECT
            year,
            month,
            total_sales,
            AVG(total_sales) OVER (ORDER BY year, month ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS rolling_avg
        FROM MonthlySales
    )

    SELECT
        year,
        month,
        total_sales,
        rolling_avg
    FROM SalesWithRollingAverage
    ORDER BY year, month;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\2383595190.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,year,month,total_sales,rolling_avg
0,2021.0,1.0,35.64,35.640000
1,2021.0,2.0,37.62,36.630000
2,2021.0,3.0,37.62,36.960000
3,2021.0,4.0,37.62,37.125000
4,2021.0,5.0,37.62,37.224000
5,2021.0,6.0,37.62,37.290000
6,2021.0,7.0,37.62,37.620000
7,2021.0,8.0,37.62,37.620000
8,2021.0,9.0,37.62,37.620000
9,2021.0,10.0,37.62,37.620000


### Insight:
1. Stable Sales with Spikes: Sales were generally stable from early 2021 to late 2022, with notable spikes in early 2022 and increased sales in early 2023.

2. Growing Sales in 2023: Sales grew significantly in 2023, with peaks in April and consistent high figures throughout the year.

3. Rolling Average Increase: The rolling average trend shows growth through 2023 and 2024, reaching its highest in mid-2024.

4. Decline in 2024-2025: A decline in sales is observed towards the end of 2024 and into early 2025.

5. Actionable Insights: Investigate causes of sales spikes and declines, plan for seasonal trends, and optimize inventory and marketing strategies based on these trends.

# Customers who have not made any purchases in the last year

In [31]:
df = pd.read_sql_query('''
    SELECT 
        c.customer_id,
        CONCAT(c.first_name, ' ', c.last_name),
        MAX(invoice_date) AS last_transaction
    FROM customer c
    JOIN invoice i ON c.customer_id = i.customer_id
    WHERE i.invoice_date < (SELECT MAX(invoice_date) FROM invoice) - INTERVAL '1' YEAR
    GROUP BY c.customer_id;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_39752\816693728.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,customer_id,concat,last_transaction
0,29,Robert Brown,2023-10-26
1,54,Steve Murray,2023-06-24
2,4,Bjørn Hansen,2024-02-27
3,34,João Fernandes,2024-10-01
4,51,Joakim Johansson,2024-12-02
5,52,Emma Jones,2023-11-08
6,10,Eduardo Martins,2024-01-09
7,35,Madalena Sampaio,2023-09-07
8,45,Ladislav Kovács,2024-11-29
9,6,Helena Holý,2024-04-11


Reach out to these customers for re-engagement